In [75]:
import numpy as np   # numerical computations
import pandas as pd    #dataframes and spreadsheets
pd.set_option('display.max_columns', 22)
import datetime as dt
from sklearn.preprocessing import LabelEncoder 
from sklearn.model_selection import train_test_split


import matplotlib.pyplot as plt      # visualisations and charts


from sqlalchemy import create_engine

In [8]:
# pip install mysql-connector-python #(option 2 for sqlalchemy)

  Using cached mysql_connector_python-8.1.0-py2.py3-none-any.whl (581 kB)
  Using cached protobuf-4.21.12-cp37-abi3-macosx_10_9_universal2.whl (486 kB)
You should consider upgrading via the '/Users/primalnotch/Desktop/loanprediction/py_env/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [9]:
def get_table_sql(query):

    host="relational.fit.cvut.cz"
    user="guest"
    password="relational"

    connection_string_mysql = f"mysql+mysqlconnector://{user}:{password}@{host}"
    
    with create_engine(connection_string_mysql).connect() as connection:
        
        df = pd.read_sql_query(query, con=connection)
    
    return df


In [30]:
loan = get_table_sql('''SELECT * FROM financial.loan;''')
acct = get_table_sql('''SELECT * FROM financial.account;''')
client = get_table_sql('''SELECT * FROM financial.client;''')
trans = get_table_sql('''SELECT * FROM financial.trans;''')
disp = get_table_sql('''SELECT * FROM financial.disp;''')

In [11]:
loan

,loan_id,account_id,date,amount,duration,payments,status
0,4959,2,1994-01-05,80952,24,3373.0,A
1,4961,19,1996-04-29,30276,12,2523.0,B
2,4962,25,1997-12-08,30276,12,2523.0,A
3,4967,37,1998-10-14,318480,60,5308.0,D
4,4968,38,1998-04-19,110736,48,2307.0,C
...,...,...,...,...,...,...,...
677,7294,11327,1998-09-27,39168,24,1632.0,C
678,7295,11328,1998-07-18,280440,60,4674.0,C
679,7304,11349,1995-10-29,419880,60,6998.0,C
680,7305,11359,1996-08-06,54024,12,4502.0,A


In [ ]:
# EDA for loan, client( create age column), merge all relevant cells together in to one DF. Preprocess the data
# trans (collapse to unique account ID/loan ID) and merge to one DF.

In [12]:
acct

,account_id,district_id,frequency,date
0,1,18,POPLATEK MESICNE,1995-03-24
1,2,1,POPLATEK MESICNE,1993-02-26
2,3,5,POPLATEK MESICNE,1997-07-07
3,4,12,POPLATEK MESICNE,1996-02-21
4,5,15,POPLATEK MESICNE,1997-05-30
...,...,...,...,...
4495,11333,8,POPLATEK MESICNE,1994-05-26
4496,11349,1,POPLATEK TYDNE,1995-05-26
4497,11359,61,POPLATEK MESICNE,1994-10-01
4498,11362,67,POPLATEK MESICNE,1995-10-14


In [13]:
client

,client_id,gender,birth_date,district_id
0,1,F,1970-12-13,18
1,2,M,1945-02-04,1
2,3,F,1940-10-09,1
3,4,M,1956-12-01,5
4,5,F,1960-07-03,5
...,...,...,...,...
5364,13955,F,1945-10-30,1
5365,13956,M,1943-04-06,1
5366,13968,M,1968-04-13,61
5367,13971,F,1962-10-19,67


In [14]:
trans

,trans_id,account_id,date,type,operation,amount,balance,k_symbol,bank,account
0,1,1,1995-03-24,PRIJEM,VKLAD,1000,1000,None,None,NaN
1,5,1,1995-04-13,PRIJEM,PREVOD Z UCTU,3679,4679,None,AB,41403269.0
2,6,1,1995-05-13,PRIJEM,PREVOD Z UCTU,3679,20977,None,AB,41403269.0
3,7,1,1995-06-13,PRIJEM,PREVOD Z UCTU,3679,26835,None,AB,41403269.0
4,8,1,1995-07-13,PRIJEM,PREVOD Z UCTU,3679,30415,None,AB,41403269.0
...,...,...,...,...,...,...,...,...,...,...
1056315,3682983,10451,1998-08-31,PRIJEM,None,62,17300,UROK,None,NaN
1056316,3682984,10451,1998-09-30,PRIJEM,None,49,13442,UROK,None,NaN
1056317,3682985,10451,1998-10-31,PRIJEM,None,34,10118,UROK,None,NaN
1056318,3682986,10451,1998-11-30,PRIJEM,None,26,8398,UROK,None,NaN


In [16]:
client.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5369 entries, 0 to 5368
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   client_id    5369 non-null   int64 
 1   gender       5369 non-null   object
 2   birth_date   5369 non-null   object
 3   district_id  5369 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 167.9+ KB


In [35]:
client['client_age']= 2023 - pd.to_datetime(client['birth_date']).dt.year

In [31]:
disp

,disp_id,client_id,account_id,type
0,1,1,1,OWNER
1,2,2,2,OWNER
2,3,3,2,DISPONENT
3,4,4,3,OWNER
4,5,5,3,DISPONENT
...,...,...,...,...
5364,13647,13955,11349,OWNER
5365,13648,13956,11349,DISPONENT
5366,13660,13968,11359,OWNER
5367,13663,13971,11362,OWNER


In [34]:
loan.status.value_counts()

status
C    403
A    203
D     45
B     31
Name: count, dtype: int64

In [49]:
df = client.merge(disp, on = 'client_id').merge(acct, on = 'account_id').merge(loan, on = 'account_id', how='left')

In [40]:
client.shape

(5369, 5)

In [42]:
loan.account_id.nunique()

682

In [50]:
df

,client_id,gender,birth_date,district_id_x,client_age,disp_id,account_id,type,district_id_y,frequency,date_x,loan_id,date_y,amount,duration,payments,status
0,1,F,1970-12-13,18,53,1,1,OWNER,18,POPLATEK MESICNE,1995-03-24,NaN,NaN,NaN,NaN,NaN,NaN
1,2,M,1945-02-04,1,78,2,2,OWNER,1,POPLATEK MESICNE,1993-02-26,4959.0,1994-01-05,80952.0,24.0,3373.0,A
2,3,F,1940-10-09,1,83,3,2,DISPONENT,1,POPLATEK MESICNE,1993-02-26,4959.0,1994-01-05,80952.0,24.0,3373.0,A
3,4,M,1956-12-01,5,67,4,3,OWNER,5,POPLATEK MESICNE,1997-07-07,NaN,NaN,NaN,NaN,NaN,NaN
4,5,F,1960-07-03,5,63,5,3,DISPONENT,5,POPLATEK MESICNE,1997-07-07,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5364,13955,F,1945-10-30,1,78,13647,11349,OWNER,1,POPLATEK TYDNE,1995-05-26,7304.0,1995-10-29,419880.0,60.0,6998.0,C
5365,13956,M,1943-04-06,1,80,13648,11349,DISPONENT,1,POPLATEK TYDNE,1995-05-26,7304.0,1995-10-29,419880.0,60.0,6998.0,C
5366,13968,M,1968-04-13,61,55,13660,11359,OWNER,61,POPLATEK MESICNE,1994-10-01,7305.0,1996-08-06,54024.0,12.0,4502.0,A
5367,13971,F,1962-10-19,67,61,13663,11362,OWNER,67,POPLATEK MESICNE,1995-10-14,7308.0,1996-12-27,129408.0,24.0,5392.0,A


In [55]:
df['status'] = df.status.fillna('E')

In [56]:
df

,client_id,gender,birth_date,district_id_x,client_age,disp_id,account_id,type,district_id_y,frequency,date_x,loan_id,date_y,amount,duration,payments,status
0,1,F,1970-12-13,18,53,1,1,OWNER,18,POPLATEK MESICNE,1995-03-24,NaN,NaN,NaN,NaN,NaN,E
1,2,M,1945-02-04,1,78,2,2,OWNER,1,POPLATEK MESICNE,1993-02-26,4959.0,1994-01-05,80952.0,24.0,3373.0,A
2,3,F,1940-10-09,1,83,3,2,DISPONENT,1,POPLATEK MESICNE,1993-02-26,4959.0,1994-01-05,80952.0,24.0,3373.0,A
3,4,M,1956-12-01,5,67,4,3,OWNER,5,POPLATEK MESICNE,1997-07-07,NaN,NaN,NaN,NaN,NaN,E
4,5,F,1960-07-03,5,63,5,3,DISPONENT,5,POPLATEK MESICNE,1997-07-07,NaN,NaN,NaN,NaN,NaN,E
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5364,13955,F,1945-10-30,1,78,13647,11349,OWNER,1,POPLATEK TYDNE,1995-05-26,7304.0,1995-10-29,419880.0,60.0,6998.0,C
5365,13956,M,1943-04-06,1,80,13648,11349,DISPONENT,1,POPLATEK TYDNE,1995-05-26,7304.0,1995-10-29,419880.0,60.0,6998.0,C
5366,13968,M,1968-04-13,61,55,13660,11359,OWNER,61,POPLATEK MESICNE,1994-10-01,7305.0,1996-08-06,54024.0,12.0,4502.0,A
5367,13971,F,1962-10-19,67,61,13663,11362,OWNER,67,POPLATEK MESICNE,1995-10-14,7308.0,1996-12-27,129408.0,24.0,5392.0,A


In [59]:
les = LabelEncoder()
df['status'] = les.fit_transform(df.status)

In [60]:
df

,client_id,gender,birth_date,district_id_x,client_age,disp_id,account_id,type,district_id_y,frequency,date_x,loan_id,date_y,amount,duration,payments,status
0,1,F,1970-12-13,18,53,1,1,OWNER,18,POPLATEK MESICNE,1995-03-24,NaN,NaN,NaN,NaN,NaN,4
1,2,M,1945-02-04,1,78,2,2,OWNER,1,POPLATEK MESICNE,1993-02-26,4959.0,1994-01-05,80952.0,24.0,3373.0,0
2,3,F,1940-10-09,1,83,3,2,DISPONENT,1,POPLATEK MESICNE,1993-02-26,4959.0,1994-01-05,80952.0,24.0,3373.0,0
3,4,M,1956-12-01,5,67,4,3,OWNER,5,POPLATEK MESICNE,1997-07-07,NaN,NaN,NaN,NaN,NaN,4
4,5,F,1960-07-03,5,63,5,3,DISPONENT,5,POPLATEK MESICNE,1997-07-07,NaN,NaN,NaN,NaN,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5364,13955,F,1945-10-30,1,78,13647,11349,OWNER,1,POPLATEK TYDNE,1995-05-26,7304.0,1995-10-29,419880.0,60.0,6998.0,2
5365,13956,M,1943-04-06,1,80,13648,11349,DISPONENT,1,POPLATEK TYDNE,1995-05-26,7304.0,1995-10-29,419880.0,60.0,6998.0,2
5366,13968,M,1968-04-13,61,55,13660,11359,OWNER,61,POPLATEK MESICNE,1994-10-01,7305.0,1996-08-06,54024.0,12.0,4502.0,0
5367,13971,F,1962-10-19,67,61,13663,11362,OWNER,67,POPLATEK MESICNE,1995-10-14,7308.0,1996-12-27,129408.0,24.0,5392.0,0


In [63]:
les.inverse_transform(df.status)   # to revert status back to letters

array(['E', 'A', 'A', ..., 'A', 'A', 'E'], dtype=object)

In [ ]:
# df['status'] = les.inverse_transform(df.status)   # to revert status back to letters

In [65]:
client_sex = {'M':0, 'F':1}
df['gender'] = df.gender.map(client_sex)

In [66]:
df

,client_id,gender,birth_date,district_id_x,client_age,disp_id,account_id,type,district_id_y,frequency,date_x,loan_id,date_y,amount,duration,payments,status
0,1,1,1970-12-13,18,53,1,1,OWNER,18,POPLATEK MESICNE,1995-03-24,NaN,NaN,NaN,NaN,NaN,4
1,2,0,1945-02-04,1,78,2,2,OWNER,1,POPLATEK MESICNE,1993-02-26,4959.0,1994-01-05,80952.0,24.0,3373.0,0
2,3,1,1940-10-09,1,83,3,2,DISPONENT,1,POPLATEK MESICNE,1993-02-26,4959.0,1994-01-05,80952.0,24.0,3373.0,0
3,4,0,1956-12-01,5,67,4,3,OWNER,5,POPLATEK MESICNE,1997-07-07,NaN,NaN,NaN,NaN,NaN,4
4,5,1,1960-07-03,5,63,5,3,DISPONENT,5,POPLATEK MESICNE,1997-07-07,NaN,NaN,NaN,NaN,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5364,13955,1,1945-10-30,1,78,13647,11349,OWNER,1,POPLATEK TYDNE,1995-05-26,7304.0,1995-10-29,419880.0,60.0,6998.0,2
5365,13956,0,1943-04-06,1,80,13648,11349,DISPONENT,1,POPLATEK TYDNE,1995-05-26,7304.0,1995-10-29,419880.0,60.0,6998.0,2
5366,13968,0,1968-04-13,61,55,13660,11359,OWNER,61,POPLATEK MESICNE,1994-10-01,7305.0,1996-08-06,54024.0,12.0,4502.0,0
5367,13971,1,1962-10-19,67,61,13663,11362,OWNER,67,POPLATEK MESICNE,1995-10-14,7308.0,1996-12-27,129408.0,24.0,5392.0,0


In [67]:
df.columns

Index(['client_id', 'gender', 'birth_date', 'district_id_x', 'client_age',
       'disp_id', 'account_id', 'type', 'district_id_y', 'frequency', 'date_x',
       'loan_id', 'date_y', 'amount', 'duration', 'payments', 'status'],
      dtype='object')

In [68]:
feature = ['gender', 'client_age','amount', 'duration']  #feature is X (input data)

In [69]:
y = 'status' #output data

In [71]:
X = df[feature]
X

,gender,client_age,amount,duration
0,1,53,NaN,NaN
1,0,78,80952.0,24.0
2,1,83,80952.0,24.0
3,0,67,NaN,NaN
4,1,63,NaN,NaN
...,...,...,...,...
5364,1,78,419880.0,60.0
5365,0,80,419880.0,60.0
5366,0,55,54024.0,12.0
5367,1,61,129408.0,24.0


In [73]:
X['amount'] = X.amount.fillna(0)
X['duration'] = X.duration.fillna(0)

/var/folders/_2/20jh237x5j3d0lwk27v5btg80000gn/T/ipykernel_1951/2228381108.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['amount'] = X.amount.fillna(0)
/var/folders/_2/20jh237x5j3d0lwk27v5btg80000gn/T/ipykernel_1951/2228381108.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['duration'] = X.duration.fillna(0)


In [74]:
X

,gender,client_age,amount,duration
0,1,53,0.0,0.0
1,0,78,80952.0,80952.0
2,1,83,80952.0,80952.0
3,0,67,0.0,0.0
4,1,63,0.0,0.0
...,...,...,...,...
5364,1,78,419880.0,419880.0
5365,0,80,419880.0,419880.0
5366,0,55,54024.0,54024.0
5367,1,61,129408.0,129408.0


In [78]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.33, random_state=42)


ValueError: Found input variables with inconsistent numbers of samples: [5369, 6]